# HW13 - Library `sqlite3`

In [1]:
import sqlite3
import pandas as pd
import sqlalchemy
from itertools import chain

## Creating a database

In [2]:
mt = pd.read_csv("genotyping_data/metadata.csv", index_col=0)

## Creating a function to save the pandas Dataframe as SQL database

In [3]:
def sqlite3_pd_to_sql(path_to_file, path_to_db):
    """
    Reads a file (csv file, with first column as index)
    and converts it to a pandas DataFrame
    Converts this pandas DataFrame to SQL using pandas and sqlite3
    @param path_to_file: str, path to the csv file
    @param path_to_db: str, path to the output SQL database
    @return: None
    """
    # Reading the csv through pandas
    df = pd.read_csv(path_to_file, index_col=0)

    # Creating connection and database
    con = sqlite3.connect(path_to_db)

    with con:
        # Converting pd dataframe to SQL
        df.to_sql(name=path_to_db, con=con, if_exists='replace', index=False)  # one could also append here


In [4]:
sqlite3_pd_to_sql("genotyping_data/metadata.csv", "sqlite3_metadata.db")

In [5]:
sqlite3_pd_to_sql("genotyping_data/genstudio.csv", "sqlite3_genstudio.db")

<ipython-input-5-ec7cebe34cfd>:1: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  sqlite3_pd_to_sql("genotyping_data/genstudio.csv", "sqlite3_genstudio.db")
/home/annatoidze/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/annatoidze/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


Another option would be to actually use another library, `sqlalchemy`.

In [6]:
def sqlalchemy_pd_to_sql(path_to_file, absolute_path_to_db):
    """
    Reads a file (csv file, with first column as index)
    and converts it to a pandas DataFrame
    Converts this pandas DataFrame to SQL using pandas and sqlalchemy
    @param path_to_file: str, path to the csv file
    @param absolute_path_to_db: str, absolute path to the output SQL database
    @return: None
    """
    # Reading the csv through pandas
    df = pd.read_csv(path_to_file, index_col=0)

    # Create a reference for SQL library
    engine = sqlalchemy.create_engine("sqlite:////" + absolute_path_to_db)

    # Conversion to SQL
    df.to_sql(absolute_path_to_db, con=engine, if_exists='replace')

In [7]:
sqlalchemy_pd_to_sql("genotyping_data/metadata.csv", "/home/annatoidze/Documents/BI_2021_Python/hw13/sqlalchemy_metadata.db")

In [8]:
sqlalchemy_pd_to_sql("genotyping_data/genstudio.csv", "/home/annatoidze/Documents/BI_2021_Python/hw13/sqlalchemy_genstudio.db")

<ipython-input-8-3ab765753474>:1: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  sqlalchemy_pd_to_sql("genotyping_data/genstudio.csv", "/home/annatoidze/Documents/BI_2021_Python/hw13/sqlalchemy_genstudio.db")


The problem in the provided databases is that fact, that the column names are separated by whitespaces, which is generally <font color='red'>advised against</font>.

## Creating an empty SQL database with the same column names and types as a given pandas database 

SQL and python data types interconversion, respectively ([soure](https://www.w3schools.com/python/python_datatypes.asp)):

* Text Type:	str
* Numeric Types:	int, float, complex
* Sequence Types:	list, tuple, range
* Mapping Type:	dict
* Set Types:	set, frozenset
* Boolean Type:	bool
* Binary Types:	bytes, bytearray, memoryview


I searched the internet for a way to automate the process of creating the SQL database with `?` or dictionary with customizable names. Apparently, it's not really possible (without format string) and one method to actually avoid SQL injection is a function similar to `scrub` [here](https://stackoverflow.com/questions/3247183/variable-table-name-in-sqlite/3247553#3247553).

Here we can specify the table name and correct the column names with whitespaces.

In [9]:
def scrub(name):
    """
    Tries to prevent SQL injection, needed for formatting of query strings
    @param name: str, column/table name
    @return: str, without characters other than digits, characters and _
    """
    # attributation: https://stackoverflow.com/a/3247553/7505395
    return ''.join(chr for chr in name if chr.isalnum() or chr == "_")

In [15]:
# Creating a database with pandas columns and types
def pd_to_cus_sql(path_to_file, path_to_db, table_name):
    """
    Reads a file (csv file, with first column as index)
    and converts it to a pandas DataFrame
    The pandas DataFrame is converted to SQL database,
    with specified table name as well as column names
    without characters other than digits, characters and _
    @param path_to_file: str, path to the csv file
    @param path_to_db: str, path to the output SQL database
    @return: None
    """
    # Type interconversion
    types_pd_to_sql = {
        "O": "text",
        "i": "int",
        "f": "float",
    }

    # Reading the csv through pandas
    df = pd.read_csv(path_to_file, index_col=0)

    # Creating connection and database
    con = sqlite3.connect(path_to_db)

    # Cursor
    cur = con.cursor()

    # Saving column names and types as in SQL
    cols = tuple(map(scrub, tuple(df.columns)))
    types = tuple(map(lambda x: types_pd_to_sql[x.kind], df.dtypes))

    # Tuple of cols and types (col0, type0, col1, type1...)
    l = tuple(chain.from_iterable(tuple(zip(cols, types))))

    # Str for creation
    creation = f"CREATE TABLE IF NOT EXISTS {scrub(table_name)} (" + ("{} {}, " * (len(types))).format(*l)[:-2] + ")"

    # Execution of creation
    with con:
        cur.execute(creation)

        # Str for insertion
        insertion = f"INSERT INTO {scrub(table_name)} (" + ("{}," * (len(cols))).format(*cols)[
                                                           :-1] + ") values(" + ("?," * len(cols))[:-1] + ")"

        # Insert Dataframe into SQL Server row by row - this takes a lot of time due to iteration
        for index, row in df.iterrows():
            cur.execute(insertion, tuple(row.values))


In [16]:
pd_to_cus_sql("genotyping_data/metadata.csv", "cus_metadata.db", "metadata")

In [17]:
pd_to_cus_sql("genotyping_data/genstudio.csv", "cus_gen.db", "genstudio")

<ipython-input-17-75ee8b3d8d94>:1: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  pd_to_cus_sql("genotyping_data/genstudio.csv", "cus_gen.db", "genstudio")


# Create an empty SQL database with specific column names and types

We have the same problem - SQL injection, which is why I am going to use the function `scrub` here again.

In [18]:
def empty_sql_col(path_to_db, col_names, types, table_name):
    """
    Create an empty SQL database with specified column names and types,
    @param path_to_db: str, path to the output SQL database
    @param col_names: list, column names
    @param types: list, list of types of SQL
    @param table_name: str, name of the table
    @return: None, creates an empty database
    """
    # Creating connection and database
    con = sqlite3.connect(path_to_db)

    # Cursor
    cur = con.cursor()

    # Tuple of cols and types (col0, type0, col1, type1...)
    l = tuple(chain.from_iterable(tuple(zip(col_names, types))))

    # Str for creation
    creation = f"CREATE TABLE IF NOT EXISTS {scrub(table_name)} (" + (
                                                                             "{} {}, " * (len(types))).format(
        *map(scrub, l))[:-2] + ")"

    # Execution of creation
    with con:
        cur.execute(creation)

In [19]:
empty_sql_col("employee.db", ["first", "last", "pay"], ["text", "text", "float"], "employee")

# SELECTION - Let's try to filter the data that we already have

In [24]:
con = sqlite3.connect("sqlite3_genstudio.db")

In [25]:
cur = con.cursor()

### What if we have a database and we don't know the table name or column names (as it might be different from the file name)

In [26]:
cur.execute("SELECT * FROM sqlite_master WHERE type='table'")
cur.fetchall()

[('table',
  'sqlite3_genstudio.db',
  'sqlite3_genstudio.db',
  2,
  'CREATE TABLE "sqlite3_genstudio.db" (\n"SNP Name" TEXT,\n  "SNP Index" INTEGER,\n  "SNP Aux" INTEGER,\n  "Sample ID" TEXT,\n  "SNP" TEXT,\n  "Allele1 - Top" TEXT,\n  "Allele2 - Top" TEXT,\n  "Allele1 - Forward" TEXT,\n  "Allele2 - Forward" TEXT,\n  "Allele1 - AB" TEXT,\n  "Allele2 - AB" TEXT,\n  "Chr" TEXT,\n  "Position" TEXT,\n  "GC Score" REAL,\n  "GT Score" REAL,\n  "Theta" REAL,\n  "R" REAL,\n  "B Allele Freq" REAL,\n  "Log R Ratio" REAL\n)')]

### So if we have whitespaces in columns we have to use \`col_name\` when referring to them

In [27]:
all_a_chr1 = cur.execute("SELECT * FROM 'sqlite3_genstudio.db' WHERE `Allele1 - Top`='A' AND Chr=1").fetchall()

In [28]:
all_a_chr1

[('1_10673082',
  2,
  0,
  '202341831114R01C01',
  '[T/C]',
  'A',
  'A',
  'T',
  'T',
  'A',
  'A',
  '1',
  '10673082',
  0.8272,
  0.8076,
  0.039,
  0.968,
  0.0,
  0.3017),
 ('1_10723065',
  3,
  0,
  '202341831114R01C01',
  '[A/G]',
  'A',
  'A',
  'T',
  'T',
  'A',
  'A',
  '1',
  '10723065',
  0.8316,
  0.8107,
  0.011,
  1.577,
  0.0,
  0.0388),
 ('1_11337555',
  4,
  0,
  '202341831114R01C01',
  '[A/G]',
  'A',
  'A',
  'T',
  'T',
  'A',
  'A',
  '1',
  '11337555',
  0.3781,
  0.7925,
  0.045,
  1.104,
  0.0,
  0.2761),
 ('1_11426075',
  6,
  0,
  '202341831114R01C01',
  '[T/C]',
  'A',
  'A',
  'T',
  'T',
  'A',
  'A',
  '1',
  '11426075',
  0.9043,
  0.8675,
  0.034,
  0.912,
  0.0,
  0.1083),
 ('1_11445382',
  7,
  0,
  '202341831114R01C01',
  '[T/C]',
  'A',
  'A',
  'T',
  'T',
  'A',
  'A',
  '1',
  '11445382',
  0.2429,
  0.6461,
  0.176,
  2.5980000000000003,
  0.0021,
  -0.0305),
 ('1_13996200',
  9,
  0,
  '202341831114R01C01',
  '[T/C]',
  'A',
  'A',
  'T',
 

Here is the database where we deleted the whitespaces in the column names:

In [29]:
con = sqlite3.connect("cus_gen.db")

cur = con.cursor()

cur.execute("SELECT * FROM sqlite_master WHERE type='table'")
cur.fetchall()

[('table',
  'genstudio',
  'genstudio',
  2,
  'CREATE TABLE genstudio (SNPName text, SNPIndex int, SNPAux int, SampleID text, SNP text, Allele1Top text, Allele2Top text, Allele1Forward text, Allele2Forward text, Allele1AB text, Allele2AB text, Chr text, Position text, GCScore float, GTScore float, Theta float, R float, BAlleleFreq float, LogRRatio float)')]

In [49]:
# So if we have whitespaces in columns we have to use `col_name` when referring to them

all_a_chr1_gc = cur.execute("SELECT SampleID, SNPName, SNP, Chr, Position FROM genstudio WHERE Allele1Top='A' AND Chr=2 AND GCScore>0.9").fetchall()

In [50]:
all_a_chr1_gc

[('202341831114R01C01', '2_11374940', '[A/C]', '2', '11374940'),
 ('202341831114R01C01', '2_115215605', '[T/C]', '2', '115215605'),
 ('202341831114R01C01', '2_115578347', '[T/G]', '2', '115578347'),
 ('202341831114R01C01', '2_141011695', '[T/C]', '2', '141011695'),
 ('202341831114R01C01', '2_141139095', '[A/G]', '2', '141139095'),
 ('202341831114R01C01', '2_149779489', '[A/G]', '2', '149779489'),
 ('202341831114R01C01', '2_150242691', '[T/C]', '2', '150242691'),
 ('202341831114R01C01', '2_150381092', '[T/G]', '2', '150381092'),
 ('202341831114R01C01', '2_19143702', '[A/C]', '2', '19143702'),
 ('202341831114R01C01', '2_26446884', '[T/C]', '2', '26446884'),
 ('202341831114R01C01', '2_46757158', '[A/G]', '2', '46757158'),
 ('202341831114R01C01', '2_6763542', '[T/C]', '2', '6763542'),
 ('202341831114R01C01', 'ALGA0011362', '[A/G]', '2', '2633346'),
 ('202341831114R01C01', 'ALGA0011482', '[T/G]', '2', '4859812'),
 ('202341831114R01C01', 'ALGA0011566', '[T/C]', '2', '5939643'),
 ('2023418311

# I noticed that the `dna_chip_id` column in `metadata.csv` resembles the `Sample ID` in `genstudio.csv`

In [44]:
mt = pd.read_csv("genotyping_data/metadata.csv")
gen = pd.read_csv("genotyping_data/genstudio.csv")

/home/annatoidze/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [52]:
mt["dna_chip_id"]

0      202290551164R09C01
1      202341831114R02C01
2      202341831114R03C01
3      202341831114R04C01
4      202290551140R01C01
              ...        
836    203071331169R10C02
837    203071331169R12C02
838    202341831114R01C01
839    202341831114R06C01
840    202341831127R04C02
Name: dna_chip_id, Length: 841, dtype: object

In [69]:
gen['Sample ID']

0          202341831114R01C01
1          202341831114R01C01
2          202341831114R01C01
3          202341831114R01C01
4          202341831114R01C01
                  ...        
1999995    202341831127R03C02
1999996    202341831127R03C02
1999997    202341831127R03C02
1999998    202341831127R03C02
1999999    202341831127R03C02
Name: Sample ID, Length: 2000000, dtype: object

In [72]:
mer = pd.merge(mt, gen, left_on='dna_chip_id', right_on='Sample ID')
mer.to_csv("genotyping_data/merged.csv")

In [67]:
len(mer['Sample ID'].unique())

30

In [68]:
len(mer['dna_chip_id'].unique())

30

Let's save this dataframe as a separate `merged.db`.

In [73]:
sqlite3_pd_to_sql("genotyping_data/merged.csv", "merged.db")

<ipython-input-73-b59dccb0437f>:1: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  sqlite3_pd_to_sql("genotyping_data/merged.csv", "merged.db")
/home/annatoidze/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/home/annatoidze/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [105]:
con = sqlite3.connect("merged.db")

cur = con.cursor()

cur.execute("SELECT `Sample ID`, `SNP Name`, sex, breed FROM 'merged.db'").fetchall()

[('202341831114R02C01', '1_10573221', 'Хр', 'Д'),
 ('202341831114R02C01', '1_10673082', 'Хр', 'Д'),
 ('202341831114R02C01', '1_10723065', 'Хр', 'Д'),
 ('202341831114R02C01', '1_11337555', 'Хр', 'Д'),
 ('202341831114R02C01', '1_11407894', 'Хр', 'Д'),
 ('202341831114R02C01', '1_11426075', 'Хр', 'Д'),
 ('202341831114R02C01', '1_11445382', 'Хр', 'Д'),
 ('202341831114R02C01', '1_135128255', 'Хр', 'Д'),
 ('202341831114R02C01', '1_13996200', 'Хр', 'Д'),
 ('202341831114R02C01', '1_142535524', 'Хр', 'Д'),
 ('202341831114R02C01', '1_14638936', 'Хр', 'Д'),
 ('202341831114R02C01', '1_161891709', 'Хр', 'Д'),
 ('202341831114R02C01', '1_17346505', 'Хр', 'Д'),
 ('202341831114R02C01', '1_17537210', 'Хр', 'Д'),
 ('202341831114R02C01', '1_1901947', 'Хр', 'Д'),
 ('202341831114R02C01', '1_242598', 'Хр', 'Д'),
 ('202341831114R02C01', '1_2463520', 'Хр', 'Д'),
 ('202341831114R02C01', '1_2812415', 'Хр', 'Д'),
 ('202341831114R02C01', '1_286337402', 'Хр', 'Д'),
 ('202341831114R02C01', '1_294072400', 'Хр', 'Д'),
